# **Pràctica 4**

# Entrenament de models de Word2Vec

In [16]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [21]:
!pip install datasets

     ---------------------------------------- 0.0/542.0 kB ? eta -:--:--
     ------------- ------------------------ 194.6/542.0 kB 3.9 MB/s eta 0:00:01
     ----------------------------------- -- 501.8/542.0 kB 6.3 MB/s eta 0:00:01
     -------------------------------------- 542.0/542.0 kB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/25.9 MB ? eta -:--:--
     ---------------------------------------- 0.2/25.9 MB 5.0 MB/s eta 0:00:06
      --------------------------------------- 0.6/25.9 MB 6.2 MB/s eta 0:00:05
     - -------------------------------------- 0.9/25.9 MB 6.1 MB/s eta 0:00:05
     - -------------------------------------- 1.2/25.9 MB 6.2 MB/s eta 0:00:04
     -- ------------------------------------- 1.4/25.9 MB 6.0 MB/s eta 0:00:05
     -- ------------------------------------- 1.7/25.9 MB 6.0 MB/s eta 0:00:04
     --- ------------------------------------ 2.0/25.9 MB 6.0 MB/s eta 0:00:05
     --- ------------------------------------ 2.2/25.9 MB 6.1 


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
from datasets import load_dataset

dataset = load_dataset("projecte-aina/catalan_general_crawling")

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for projecte-aina/catalan_general_crawling contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/catalan_general_crawling
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1016113
    })
})

In [24]:
train_dataset = dataset['train']

In [25]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 1016113
})

In [30]:
train_dataset[:5]


{'text': ['Reduïu els costos dels processos administratius al vostre organisme públic\nEviteu els desplaçaments i pèrdua de temps als ciutadans en les seves gestions\nOferiu una administració més transparent a ciutadans i empreses\nEns grans i petits experimenten aquesta transformació amb èxit, gràcies al suport de l\'AOC\nDepartament de Sistemes d\'Informació i Processos\n" Via Oberta ens ha permès fer efectiu el dret dels ciutadans a no aportar documents, eliminant paper i simplificant procediments"\n" e.FACT proporciona informació indispensable per a la realització de les auditories del registre comptable de factures de les Administracions Públiques Catalanes"\nCoordinador del departament d\'Informàtica\n"El servei VIA OBERTA és el que ha aportat majors avantatges per als ciutadans"\n"Amb l\' e-NOTUM hem escurçat els procediments en 12 dies, quasi un 40% menys!"\nCoordinadora d\'organització de persones i e-administració\n" Via Oberta ofereix millores per als ciutadans al no haver d

In [26]:
print(train_dataset)


Dataset({
    features: ['text'],
    num_rows: 1016113
})


In [31]:
import os

def dividir_dataset(dataset, output_dir, tamano_partes):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    total_bytes = 0
    contador = 1
    current_size = 0
    current_part = []
    
    for i, row in enumerate(dataset):
        text = row['text']
        current_size += len(text.encode('utf-8'))
        current_part.append(text)
        
        if current_size >= tamano_partes[contador - 1]:
            with open(os.path.join(output_dir, f'parte_{contador}.txt'), 'w', encoding='utf-8') as f:
                for line in current_part:
                    f.write(line + '\n')
            current_part = []
            current_size = 0
            contador += 1
            
            if contador > len(tamano_partes):
                break

    # Guardar la última parte si quedó algo
    if current_part:
        with open(os.path.join(output_dir, f'parte_{contador}.txt'), 'w', encoding='utf-8') as f:
            for line in current_part:
                f.write(line + '\n')

# Tamaños deseados para cada parte (en bytes): 100MB, 500MB, 1GB
tamano_partes = [100 * 1024 * 1024, 500 * 1024 * 1024, 1 * 1024 * 1024 * 1024]
output_dir = 'divided_datasets'

# Dividir el dataset
dividir_dataset(train_dataset, output_dir, tamano_partes)


In [ ]:
import re
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from gensim.models.word2vec import LineSentence

# Función de preprocesamiento
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    tokens = word_tokenize(text)
    return tokens

# Ruta al directorio con las partes del dataset
dataset_parts = [f'divided_datasets/parte_{i}.txt' for i in range(1, len(tamano_partes) + 1)]

In [ ]:


# Entrenar un modelo Word2Vec para cada parte del dataset
for i, part in enumerate(dataset_parts):
    # Leer y preprocesar el archivo
    sentences = LineSentence(part)
    
    # Entrenar el modelo Word2Vec
    model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=10, workers=4, sg=1, epochs=5)
    
    # Guardar el modelo
    model.save(f'word2vec_model_part_{i+1}.model')

    print(f'Model for part {i+1} trained and saved.')


Per entranar el model Word2Vec, ho fem amb diferents mides del Dataset (100MB, 500MB, 1GB, complet). Per fer aquesta divisió, definim la funció dividir_archivo, aquesta funció reb com a paràmetres la ruta de l'arxiu i la mida de les diferents parts (en Bytes), y genera aquests fitxers en la mateixa ruta que l'original.

In [11]:
def dividir_archivo(ruta_archivo, tamano_partes):
    # Abrir el archivo en modo lectura binaria
    with open(ruta_archivo, 'rb') as archivo_original:
        contador = 1
        for tamano_parte in tamano_partes:
            with open(ruta_archivo + f"_parte{contador}.txt", 'wb') as archivo_parte:
                bytes_leidos = 0
                while bytes_leidos < tamano_parte:
                    # Leer un bloque del tamaño especificado o hasta el final del archivo
                    contenido = archivo_original.read(min(4096, tamano_parte - bytes_leidos))
                    if not contenido:
                        break  # Si no hay más contenido, terminar
                    archivo_parte.write(contenido)
                    bytes_leidos += len(contenido)
            contador += 1

# Ruta del archivo a dividir
ruta_archivo = 'C:/Users/Usuario/Desktop/uni/Q4/PLH/pract4/catalan_general_crawling/corpus/catalan_general_crawling.txt'
# Tamaños deseados para cada parte (en bytes)
tamano_partes = [100 * 1024 * 1024, 500 * 1024 * 1024, 1 * 1024 * 1024 * 1024]  # 100 MB, 500 MB, 1 GB

# Llamar a la función para dividir el archivo
dividir_archivo(ruta_archivo, tamano_partes)


Una vegada tenim les diferents parts creades, passem a preprocessar el text per tal de 

In [19]:
import nltk
import spacy
import string

# Descargar recursos necesarios de NLTK
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Cargar el modelo de spaCy para catalán
nlp = spacy.load('ca_core_news_sm')

# Función para preprocesar el texto
def preprocesar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()
    
    # Tokenizar el texto
    tokens = word_tokenize(texto)
    
    # Eliminar puntuación y stopwords
    stop_words = set(stopwords.words('catalan'))
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
    
    # Lemmatización con spaCy
    doc = nlp(' '.join(tokens))
    tokens = [token.lemma_ for token in doc]
    
    return tokens

# Función para preprocesar el archivo
def preprocesar_archivo(ruta_archivo, ruta_salida):
    with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
        with open(ruta_salida, 'w', encoding='utf-8') as archivo_salida:
            for linea in archivo:
                tokens = preprocesar_texto(linea)
                archivo_salida.write(' '.join(tokens) + '\n')

# Preprocesar cada parte del archivo dividido
partes = ["parte1", "parte2", "parte3"]
for parte in partes:
    ruta_parte = f'C:/Users/Usuario/Desktop/uni/Q4/PLH/pract4/catalan_general_crawling/corpus/catalan_general_crawling.txt_{parte}.txt'
    ruta_salida = f'C:/Users/Usuario/Desktop/uni/Q4/PLH/pract4/catalan_general_crawling/corpus/catalan_general_crawling_preprocessed_{parte}.txt'
    preprocesar_archivo(ruta_parte, ruta_salida)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KeyboardInterrupt: 

In [ ]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

# Entrenar el modelo Word2Vec con cada parte preprocesada
for parte in partes:
    corpus_file = f'C:/Users/Usuario/Desktop/uni/Q4/PLH/pract4/catalan_general_crawling/corpus/catalan_general_crawling_preprocessed_{parte}.txt'
    sentences = LineSentence(corpus_file)
    model = Word2Vec(sentences, vector_size=100, window=5, min_count=10, workers=4, epochs=25)
    model.save(f'word2vec_model_{parte}.model')

per cada frase un unic vector
TF-IDF per descartar paraules uq no aporten info

In [ ]:
import tensorflow as tf
def build_and_compile_model(hidden_size: int = 64) -> tf.keras.Model:
  model = tf.keras.Sequential([
      tf.keras.layers.Concatenate(axis=-1, ),
      tf.keras.layers.Dense(hidden_size, activation='relu'),
      tf.keras.layers.Dense(1)
  ])
  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model
m = build_and_compile_model()
# E.g.
import numpy as np
y = m((np.ones((1, 100)), np.ones((1,100)), ), )

el primer 10 s'ha de canviar per la long maxima del vector d'entrada

In [ ]:
import tensorflow as tf
def build_and_compile_model(
        input_length: int = 10, hidden_size: int = 64, dictionary_size: int = 1000, embedding_size: int = 16,
) -> tf.keras.Model:
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32, ), tf.keras.Input((input_length, ), dtype=tf.int32, )
    # Define Layers
    embedding = tf.keras.layers.Embedding(
        dictionary_size, embedding_size, input_length=input_length, mask_zero=True, )
    pooling = tf.keras.layers.GlobalAveragePooling1D()
    concatenate = tf.keras.layers.Concatenate(axis=-1, )
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)
    # Pass through the layers
    _input_mask_1, _input_mask_2 = tf.not_equal(input_1, 0), tf.not_equal(input_2, 0)
    _embedded_1, _embedded_2 = embedding(input_1, ), embedding(input_2, )
    _pooled_1, _pooled_2 = pooling(_embedded_1, mask=_input_mask_1), pooling(_embedded_2, mask=_input_mask_2)
    _concatenated = concatenate((_pooled_1, _pooled_2, ))
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)
    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2, ), outputs=_output, )
    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
    return model